# **Using Physical Attributes and Point Percentages to Predict Position for NBA Players**

##### *Tanisha Amrin, Charmaine Chui, Jakob Sereda, Julian Wright*

In [5]:
# importing libraries 
library(tidyverse)
library(tidymodels)
library(RColorBrewer)

In [6]:
# reading in the data from the web
nba_data <- read_csv("https://raw.githubusercontent.com/jakobsereda/dsci-100-project/main/data/stats.csv", show_col_types = FALSE) 

In [7]:
# cleaning and wrangling the data
names(nba_data) <- tolower(names(nba_data))

nba_clean <- nba_data |>
    select(person_id, first_name, last_name, height, weight, position, season_id, 
           gp, gs, min, fg_pct, fg3_pct, ft_pct, reb, ast, stl, blk, tov, pf, pts) |>
    na.omit() |>
    mutate(position = ifelse(position == "Center-Forward", "Center", position)) |>
    mutate(position = ifelse(position == "Guard-Forward", "Guard", position)) |>
    mutate(position = ifelse(position == "Forward-Guard" | position == "Forward-Center", "Forward", position)) |>
    mutate(position = as_factor(position))

In [11]:
set.seed(2024) 

# splitting the data into training and testing sets
nba_split <- initial_split(nba_clean, prop = 3/4, strata = position)

nba_train <- training(nba_split)
nba_test <- testing(nba_split)

head(nba_train)

person_id,first_name,last_name,height,weight,position,season_id,gp,gs,min,fg_pct,fg3_pct,ft_pct,reb,ast,stl,blk,tov,pf,pts
<dbl>,<chr>,<chr>,<dbl>,<dbl>,<fct>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
76003,Kareem,Abdul-Jabbar,86,225,Center,1988-89,74,74,1695,0.475,0.000,0.739,334,74,38,85,95,196,748
201582,Alexis,Ajinca,86,248,Center,2016-17,39,15,584,0.500,0.000,0.725,177,12,20,22,31,77,207
202374,Solomon,Alabi,85,252,Center,2011-12,14,0,122,0.361,0.000,0.875,47,3,2,9,5,11,33
202332,Cole,Aldrich,83,253,Center,2017-18,21,0,49,0.333,0.000,0.333,15,3,2,1,1,11,12
200746,LaMarcus,Aldridge,83,250,Center,2021-22,47,12,1050,0.550,0.304,0.873,258,42,14,47,44,78,607
76021,Chuck,Aleksinas,83,260,Center,1984-85,74,4,1114,0.478,0.000,0.733,270,36,15,15,72,171,377


In [12]:
# summarizing the number of players in each position, and the min, max, and mean of each predictor for each position
nba_summarized <- nba_train |>
    group_by(position) |>
    summarize(min_height = min(height), max_height = max(height), mean_height = mean(height),
              min_weight = min(weight), max_weight = max(weight), mean_weight = mean(weight),
              min_fg_pct = min(fg_pct), max_fg_pct = max(fg_pct), mean_fg_pct = mean(fg_pct),
              min_ft_pct = min(ft_pct), max_ft_pct = max(ft_pct), mean_ft_pct = mean(ft_pct),
              n_players = n())

nba_summarized

position,min_height,max_height,mean_height,min_weight,max_weight,mean_weight,min_fg_pct,max_fg_pct,mean_fg_pct,min_ft_pct,max_ft_pct,mean_ft_pct,n_players
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
Forward,71,86,80.11799,153,289,226.2340,0,1,0.4092183,0,1,0.6031200,1017
Center,79,91,83.36118,195,360,250.5577,0,1,0.4317838,0,1,0.5567199,407
Guard,65,84,75.56789,133,255,194.8569,0,1,0.3732541,0,1,0.6561018,1090


In [ ]:
set.seed(2024)

# creating recipe and standardizing data
nba_recipe <- recipe(position ~ height + weight + fg_pct + ft_pct, data = nba_train) |>
    step_scale(all_predictors()) |>
    step_center(all_predictors())

# creating knn-model specification
